# Scrape Stories from Stadt Frankfurt

Scrape German Corona stories from the website of the city of Frankfurt (Stadt Frankfurt am Main): https://frankfurt.de/service-und-rathaus/presse/texte-und-kampagnen/corona-geschichten.

The stories are in an interview format with questions and answers. The stories are mostly captured in the answers, but for completeness, both questions and answers are scraped and stored.

The stories are stored in dictionaries with the following fields:

- `link`: The link of the story page (string)
- `title`: The title of the story page on the main page (string)
- `headline`: The title of the story by the author (string)
- `subline`: The subtitle of the story describing the interviewee (string)
- `intro_text`: The introductory text giving some background info on the interviewee and story (string)
- `question_text`: A list with the interview questions (strings)
- `answer_text`: A list with the interview answers (strings)
- `interviewee`: The name of the interviewee (string)

The web pages containing the stories are stored as `.html`. The notebook requires a folder at `DATA_DIR`.

In [1]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
from string import capwords

In [2]:
# Set URL of the main page containing the links to the story pages

URL = "https://frankfurt.de/service-und-rathaus/presse/texte-und-kampagnen/corona-geschichten"

# Set directory for storing web pages

DATA_DIR = "../frankfurt.de/scraped/"

In [3]:
# Function to extract links to the story pages from main page

def extract_hrefs_from_url(URL):
    main_page = BeautifulSoup(urlopen(URL).read(), "html.parser")
    
    hrefs = []

    for instance in main_page.find_all("a", attrs = {"class": "contentTile noprint"}):
        new_link = instance.get("href")
        pos_443 = new_link.index(":443")
        hrefs.append(new_link[:pos_443] + new_link[pos_443+4:])
        print("Extracted link: {}".format(hrefs[-1]))
    
    print("Done")
    
    return hrefs

In [9]:
# Function to extract text from story pages

def extract_text_from_url(URL):
    new_page = BeautifulSoup(urlopen(URL).read(), "html.parser")
    new_title = new_page.title.string[:new_page.title.string.find(" |")]
    new_headline = new_page.find("h4", attrs = {"class": "_headline"})
    new_subline = new_page.find("p", attrs = {"class": "_subline"})
    new_intro_text = new_page.find("p", attrs = {"class": "_introTxt"})
    new_inline_text = new_page.find("span", attrs = {"class": "_inlineTxt"})
    new_stripped_text = [text for text in new_inline_text.stripped_strings]
    new_question_text = [new_stripped_text[i-1] for i in range(len(new_stripped_text)) if new_stripped_text[i].isupper() and i > 0]
    new_answer_text = [text for i, text in enumerate(new_stripped_text) if new_stripped_text[i-1].isupper() and i > 0]
    new_interviewee = [capwords(text[:-1]) for text in new_stripped_text if text.isupper()]
    new_doc = {
        "link": URL,
        "title": new_title,
        "headline": new_headline.get_text() if new_headline is not None else "",
        "subline": new_subline.get_text() if new_subline is not None else "",
        "intro_text": new_intro_text.get_text() if new_intro_text is not None else "",
        "question_text": new_question_text,
        "answer_text": new_answer_text,
        "interviewee": new_interviewee[0] if len(new_interviewee) > 0 else ""
    }
    print("Extracted text from: {}".format(URL))
    
    # Save content of webpage as html for sustainability
    
    with open(DATA_DIR + URL.split("/")[-1] + ".html", "w") as file:
        print(urlopen(URL).read(), file = file)
    
    return new_doc

In [10]:
# Wrapper to for list of story pages

def extract_texts_from_url_list(URL_list):
    docs = []
    for URL in URL_list:
        docs.append(extract_text_from_url(URL))
    
    print("Done")
    
    return docs

In [11]:
# Function to print doc

def print_doc(doc):
    for field in doc.keys():
        if isinstance(doc[field], list):
            print(field + ": " + "".join([s + "\n" + "\t" for s in doc[field]]) + "\n")
        else:
            print(field + ": " + doc[field] + "\n")
        
    return

In [12]:
# Extract links from main page

hrefs = extract_hrefs_from_url(URL)

Extracted link: https://frankfurt.de/service-und-rathaus/presse/texte-und-kampagnen/corona-geschichten/die-vhs-mitarbeiterin
Extracted link: https://frankfurt.de/service-und-rathaus/presse/texte-und-kampagnen/corona-geschichten/die-kfz-zulassungsstelle
Extracted link: https://frankfurt.de/service-und-rathaus/presse/texte-und-kampagnen/corona-geschichten/das-buergeramt
Extracted link: https://frankfurt.de/service-und-rathaus/presse/texte-und-kampagnen/corona-geschichten/abteilungsleiterin-strassenverkehrsamt
Extracted link: https://frankfurt.de/service-und-rathaus/presse/texte-und-kampagnen/corona-geschichten/der-pfarrer
Extracted link: https://frankfurt.de/service-und-rathaus/presse/texte-und-kampagnen/corona-geschichten/das-amtsblatt-team
Extracted link: https://frankfurt.de/service-und-rathaus/presse/texte-und-kampagnen/corona-geschichten/katja-hartung
Extracted link: https://frankfurt.de/service-und-rathaus/presse/texte-und-kampagnen/corona-geschichten/magnus-welkerling
Extracted li

In [13]:
# Extract texts from links

docs = extract_texts_from_url_list(hrefs)

Extracted text from: https://frankfurt.de/service-und-rathaus/presse/texte-und-kampagnen/corona-geschichten/die-vhs-mitarbeiterin
Extracted text from: https://frankfurt.de/service-und-rathaus/presse/texte-und-kampagnen/corona-geschichten/die-kfz-zulassungsstelle
Extracted text from: https://frankfurt.de/service-und-rathaus/presse/texte-und-kampagnen/corona-geschichten/das-buergeramt
Extracted text from: https://frankfurt.de/service-und-rathaus/presse/texte-und-kampagnen/corona-geschichten/abteilungsleiterin-strassenverkehrsamt
Extracted text from: https://frankfurt.de/service-und-rathaus/presse/texte-und-kampagnen/corona-geschichten/der-pfarrer
Extracted text from: https://frankfurt.de/service-und-rathaus/presse/texte-und-kampagnen/corona-geschichten/das-amtsblatt-team
Extracted text from: https://frankfurt.de/service-und-rathaus/presse/texte-und-kampagnen/corona-geschichten/katja-hartung
Extracted text from: https://frankfurt.de/service-und-rathaus/presse/texte-und-kampagnen/corona-ge

In [14]:
# Show example doc

print_doc(docs[0])

link: https://frankfurt.de/service-und-rathaus/presse/texte-und-kampagnen/corona-geschichten/die-vhs-mitarbeiterin

title: Die VHS-Mitarbeiterin

headline: „Viele sind froh, dass sie sich überhaupt treffen können“

subline: Nicola Wagner, pädagogische Mitarbeiterin im Bereich Kultur, berichtet, wie Corona ihre Arbeit und den Kursalltag der VHS verändert hat.

intro_text: Auch die Erwachsenenbildung wurde durch die Pandemie und die Maßnahmen stark eingeschränkt. Betroffen ist hiervon auch die Frankfurter Volkshochschule (VHS). Die VHS Frankfurt ist mit rund 5.600 Veranstaltungen im Jahr (Kurse, Seminare, Workshops, Vorträge, Online-Angebote, Führungen, Bildungsurlaube und Ausstellungen) die größte öffentliche Weiterbildungseinrichtung in Hessen. Im Interview erzählt Nicola Wagner, pädagogische Mitarbeiterin im Bereich Kultur, wie Corona ihre Arbeit und den Kursalltag der VHS verändert hat.

question_text: Frau Wagner, beschreiben Sie bitte kurz Ihren Job bei der Volkshochschule
Frankfur